In [1]:
import sys
sys.path.append('../Codes/library/')
sys.path.append('../Codes/Python/')
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
import scipy.special as sc
import pandas as pd
from Immuno_models import*
N_A = 6.02214076e23
Text_files_path = '/Users/robertomorantovar/Dropbox/Research/Evolution_Immune_System/Text_files/'

%autosave 60

Autosaving every 60 seconds


In [2]:
N_mesas = 1.12e5
T_votos = 2.3e6

l_1 = (-np.log(0.26))
print('Promedio 1 : %.2e'%(l_1), '-->  Total votos : %.2e'%(l_1*N_mesas))

l_2 = T_votos/N_mesas
print('Promedio 2 : %.2e'%(l_2), '-->  Probabilidad de cero votos : %.2e'%(np.exp(-l_2)*100), '-->  Mesas con cero votos : %.2e'%(np.exp(-l_2)*N_mesas))


Promedio 1 : 1.35e+00 -->  Total votos : 1.51e+05
Promedio 2 : 2.05e+01 -->  Probabilidad de cero votos : 1.21e-07 -->  Mesas con cero votos : 1.35e-04


In [15]:
np.random.random()

0.2608403973986765

In [47]:
np.log(1e-12)

-27.631021115928547

In [49]:
np.exp(-24.6)

2.0718377657208856e-11

In [34]:
1e6/(N_A)*1e8*1e3

1.6605390671738467e-07

In [58]:
'%.1e'%(1e3*np.exp(1*35)/N_A)

'2.6e-06'